In [1]:
# Dependencies
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
from torchvision.datasets import MNIST
from torchvision import transforms
from torch import optim
from torch import nn
import torch

from src.transforms import AddGaussianNoise
from src.transforms import AddSaltPepperNoise
from src.transforms import AddBlockNoise
from src.transforms import ClampTensor

from src.autoencoder import Autoencoder

import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

%matplotlib inline

In [2]:
# Define random seed
RANDOM_SEED = 42

# Set random seed for numpy
_ = np.random.seed(RANDOM_SEED)
# Set random seed for pytorch
_ = torch.random.manual_seed(RANDOM_SEED)

In [3]:
# Get devices
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cpu = torch.device('cpu')

## Dataset creation

In [4]:
# MNIST dataset
MNIST_DIR_PATH = './data'  # MNIST dataset directory
MNIST_DOWNLOAD = True  # Enable MNIST download/overwrite

In [5]:
# Load/download MNIST dataset

# Define a simple transformer pipeline for test dataset
test_transform = transforms.Compose([
    transforms.ToTensor(),
    ClampTensor()
])

# Test dataset
test_dataset = MNIST(MNIST_DIR_PATH, train=False, download=MNIST_DOWNLOAD, transform=test_transform)

# Check training dataset shape
print('Test dataset has %d images' % len(test_dataset))

# Show some sample of training dataset
fig, axs = plt.subplots(1, 7, figsize=(14, 2))
# Loop through each subplot
for ax in axs.flatten():
    # Get an image and the corresponding label
    img, label = test_dataset[np.random.choice(len(test_dataset))]
    ax.imshow(img.squeeze().numpy(), cmap='gist_gray')  # Show grayscale image
    ax.set_title('Label: %d' % label)  # Show label in title
    ax.set_xticks([])  # Remove x-axis ticks
    ax.set_yticks([])  # Remove y-axis ticks
plt.tight_layout()

NameError: name 'train_dataset' is not defined

In [ ]:
# Train dataset: it is a concatenation of either original and noised datasets
train_dataset_list = [
    # Original MNIST training dataset
    MNIST(MNIST_DIR_PATH, train=True, download=MNIST_DOWNLOAD, transform=transforms.Compose([
        transforms.ToTensor(),
        ClampTensor()
    ])),
    # Gaussian noised MNIST training dataset
    MNIST(MNIST_DIR_PATH, train=True, download=MNIST_DOWNLOAD, transform=transforms.Compose([
        transforms.ToTensor(),
        AddGaussianNoise(mean=0.0, std=0.5),
        ClampTensor()
    ])),
    # Salt and pepper noised MNIST training dataset
    MNIST(MNIST_DIR_PATH, train=True, download=MNIST_DOWNLOAD, transform=transforms.Compose([
        transforms.ToTensor(),
        AddSaltPepperNoise(percentage=0.6),
        ClampTensor()
    ])),
    # Block noised MNIST training dataset
    MNIST(MNIST_DIR_PATH, train=True, download=MNIST_DOWNLOAD, transform=transforms.Compose([
        transforms.ToTensor(),
        AddBlockNoise(percentage=0.45),
        ClampTensor()
    ]))
]

# Check training dataset shape
print('Default training dataset has {:d} images'.format(len(train_dataset_list[0])))
print('Therefore, in training dataset with noise there will be {:d} x {:d} = {:d} images'.format(
    len(train_dataset_list[0]), len(train_dataset_list), 
    len(train_dataset_list[0]) * len(train_dataset_list)
))

# Show some sample of training datasets
fig, axs = plt.subplots(4, 7, figsize=(14, 8))
# Loop through each row
for i in range(axs.shape[0]):
    # Define current dataset
    curr_dataset = train_dataset_list[i]
    # Get current dataset length
    n = len(curr_dataset)
    # Loop through each column
    for j in range(axs.shape[1]):
        # Get an image and the corresponding label from current dataset
        img, label = curr_dataset[np.random.choice(n)]
        # Show grayscale image
        axs[i, j].imshow(img.squeeze().numpy(), cmap='gist_gray')
        axs[i, j].set_title('Label: %d' % label)  # Show label in title
        axs[i, j].set_xticks([])  # Remove x-axis ticks
        axs[i, j].set_yticks([])  # Remove y-axis ticks
# Show plot
plt.tight_layout()

In [ ]:
# Create a single training dataset instance
train_dataset = ConcatDataset(train_dataset_list)

# Check training dataset shape
print('Training dataset with noise has %d images' % len(train_dataset))

# Show some sample of training dataset
fig, axs = plt.subplots(1, 7, figsize=(14, 2))
# Loop through each subplot
for ax in axs.flatten():
    # Get an image and the corresponding label
    img, label = train_dataset[np.random.choice(len(train_dataset))]
    ax.imshow(img.squeeze().numpy(), cmap='gist_gray')  # Show grayscale image
    ax.set_title('Label: %d' % label)  # Show label in title
    ax.set_xticks([])  # Remove x-axis ticks
    ax.set_yticks([])  # Remove y-axis ticks
plt.tight_layout()

## Autoencoder

In [ ]:
# Initialize a new simple autoencoder by first initializing its components

# Convolutional encoder
encoder_cnn = nn.Sequential(
    nn.Conv2d(1, 8, 3, stride=2, padding=1),
    nn.ReLU(True),
    nn.Conv2d(8, 16, 3, stride=2, padding=1),
    nn.ReLU(True),
    nn.Conv2d(16, 32, 3, stride=2, padding=0),
    nn.ReLU(True)
)

# Test convolutional encoder: get convolutional to linear transformations
out = encoder_cnn(torch.rand((1, 1, 28, 28), dtype=torch.float))
# Print out shape
print('Last convolutional layer has shape %s' % str(tuple(out.size())))

In [ ]:
# Linear encoder
encoder_lin = nn.Sequential(
    # Set first linear layer according to last cnn layer (3 * 3 * 32)
    nn.Linear(3 * 3 * 32, 64),  
    nn.ReLU(True),
    # Define last encoder linear layer size (16)
    nn.Linear(64, 16)
)

# Linear decoder
decoder_lin = nn.Sequential(
    # Set first decoder linear layer according to last encoder linear layer 
    nn.Linear(16, 64),
    nn.ReLU(True),
    # Set decoder last linear layer specularly to first encoder linear layer 
    nn.Linear(64, 3 * 3 * 32),
    nn.ReLU(True)
)

# Convolutional decoder
decoder_cnn = nn.Sequential(
    # Set decoder first cnn layer specularly to last encoder cnn layer
    nn.ConvTranspose2d(32, 16, 3, stride=2, output_padding=0),
    nn.ReLU(True),
    nn.ConvTranspose2d(16, 8, 3, stride=2, padding=1, output_padding=1),
    nn.ReLU(True),
    nn.ConvTranspose2d(8, 1, 3, stride=2, padding=1, output_padding=1)
)

# Generate the autoencoder
net = Autoencoder(encoder_cnn, encoder_lin, decoder_lin, decoder_cnn, lin_to_cnn = (32, 3, 3))
net

## Train and test

In [ ]:
# Initialize dataloaders

# Define training dataloader settings
BATCH_SIZE=512
SHUFFLE = True

# Initialize train dataloader
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
# Initialize test dataloader
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [ ]:
# out, loss = net.test_batch(
#     batch=next(iter(test_dataloader)), 
#     loss_fn=nn.MSELoss()
# )
# 
# loss.item()

In [ ]:
# Iteratively train and test the autoencoder

# Define number of epochs
NUM_EPOCHS = 100
LINEAR_RATE = 1e-3
WEIGHT_DECAY = 1e-5

# Define loss function
loss_fn = nn.MSELoss()

# Define optimizer
optimizer = optim.Adam(net.parameters(), lr=LINEAR_RATE, weight_decay=WEIGHT_DECAY)

# Execute training
train_loss, test_loss = net.train_epochs(
    num_epochs=NUM_EPOCHS, train_dataloader=train_dataloader, test_dataloader=test_dataloader,
    optimizer=optimizer, loss_fn=loss_fn, device=device, verbose=True
)

# Plot losses
fig, ax = plt.subplots(1, 1, figsize=(14, 7))
_ = ax.set_title('Train vs test loss')
_ = ax.plot(x=range(0, len(train_loss)), y=train_loss)
_ = ax.plot(x=range(0, len(train_loss), len(train_loss) / NUM_EPOCHS), y=test_loss)
_ = plt.show()